## Symbolic analysis of models

In this notebook, we use symbolic mathematics to study energy-based PySB model. We derive steady-state analytical solutions to study reaction networks without costly ODE simulations. First, we load the previously developed models for RAF and RAF inhibition:

In [1]:
#import complex_example_asym_RAF_two_RAFi as model
import toy_example_RAF_RAFi as model
from pysb.bng import generate_equations
from util_display import display_model_info

#generate the model equations 
model=model.model
generate_equations(model)

#display model informations
display_model_info(model)

Model information
Species: 6
Parameters: 12
Expressions: 8
Observables: 8
Total Rules: 2
Energy Rules: 2
Non-energy Rules: 0
Energy Patterns: 4
Reactions: 12


Next, we define the sympy systems of equations corresponding to the ODE system but with the left-hand side (the derivative definition) set to zero, meaning that the system is considered to be at steady-state: 

In [2]:
import sympy

#create a list of expressions to be substituted into kinetic rates of reactions
subs = {e: e.expand_expr() for e in model.expressions | model._derived_expressions}
subs.update({sympy.Symbol(f"__s{i}"): sympy.Symbol(f"s_{i}") for i in range(len(model.species))}) 
kinetics = sympy.Matrix([r['rate'] for r in model.reactions]).xreplace(subs)    
#simplyfy kinetic
kinetics.simplify()
sm = sympy.SparseMatrix(*model.stoichiometry_matrix.shape, model.stoichiometry_matrix.todok())
odes = (sm * kinetics)


Then, we define a last equations that calculates the amount of active RAFs based on the status of RAF dimerization and drug binding:

We use symbolic solvers to derive the analytical function for the amount of active RAFs at steady state: 